In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf

from PIL import Image

from keras.preprocessing.image import load_img, img_to_array
import os

from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt

from tensorflow.python.framework import ops

import math

from __future__ import division


Using TensorFlow backend.


In [ ]:
import scipy
from PIL import Image
from scipy import ndimage

my_image = '0a1f8334a9f583cac009dc033c681e47.jpg'

dir = "/home/saniya/Downloads/train/"

fname = dir + my_image

image = np.array(ndimage.imread(fname, flatten=False))

my_image = scipy.misc.imresize(image, size=(64,64)).reshape(1, 64*64*3).T

my_image.shape

In [2]:
X_train = []
def convert_to_sixty_four():
    dir = '/home/saniya/Downloads/train/'
    for filename in os.listdir(dir):
        path = dir + str(filename)
        image = np.array(ndimage.imread(path, flatten=False, mode='L'))
        my_image = scipy.misc.imresize(image, size=(28,28)).reshape(1, 28*28).T
        print '{} image shape is {}'.format(filename, my_image.shape)
        X_train.append(my_image)

In [ ]:
convert_to_sixty_four()

In [184]:
X_train = np.array(X_train)

In [185]:
X_train.shape
#Normalize image vectors
X_train = X_train/255

In [186]:
X_train = X_train.reshape(X_train.shape[1], X_train.shape[0])

In [24]:

X_train = []
def convert_to_224():
    for filename in os.listdir(dir):
        path = dir + "/" + str(filename)
        img = load_img(path, target_size=(224,224))
        imgarr = img_to_array(img)
        print '{} image shape is {}'.format(filename, imgarr.shape)
        X_train.append(imgarr)

In [9]:
def one_hot_matrix(labels, C):
    """
    Creates a one_hot matrix of the labels
    
    Arguments:
    labels -- Vector containing labels
    C -- number of classes, depth of one hot dimension
    """
    
    C = tf.constant(C, name="C")
    one_hot_matrix = tf.one_hot(labels, C, axis=0)
    sess = tf.Session()
    one_hot = sess.run(one_hot_matrix)
    sess.close()
    
    return one_hot

In [7]:
breed_labels = pd.read_csv('/home/saniya/Downloads/labels.csv')
lb = LabelEncoder()
lb.fit(breed_labels.breed)
numeric_labels = lb.transform(breed_labels.breed)
hotted = one_hot_matrix(numeric_labels, 12)

In [15]:
hotted = one_hot_matrix(numeric_labels, 120)

In [16]:
def create_placeholders(n_x, n_y):
    
    """
    Creates tensorflow placeholders for inputs and labels
    """
    
    X = tf.placeholder(tf.float32, [n_x, None], name="X")
    Y = tf.placeholder(tf.float32, [n_y, None], name="Y")
    
    return X,Y

In [188]:
def initialize_parameters():
    """
    Initializes parameters to build a neural network with tensorflow. The shapes are:
                        W1 : [25, 12288]
                        b1 : [25, 1]
                        W2 : [12, 25]
                        b2 : [12, 1]
                        W3 : [6, 12]
                        b3 : [6, 1]
    
    Returns:
    parameters -- a dictionary of tensors containing W1, b1, W2, b2, W3, b3
    """
    
    tf.set_random_seed(1)                   # so that your "random" numbers match ours
        
   
    W1 = tf.get_variable("W1", [480, 784], initializer=tf.contrib.layers.xavier_initializer(seed=1))
    b1 = tf.get_variable("b1", [480, 1], initializer=tf.zeros_initializer())
    W2 = tf.get_variable("W2", [240,480], initializer=tf.contrib.layers.xavier_initializer(seed=1))
    b2 = tf.get_variable("b2", [240,1], initializer=tf.zeros_initializer())
    W3 = tf.get_variable("W3", [120,240], initializer=tf.contrib.layers.xavier_initializer(seed=1))
    b3 = tf.get_variable("b3", [120, 1], initializer=tf.zeros_initializer())
    

    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2,
                  "W3": W3,
                  "b3": b3}
    
    return parameters

In [189]:
def forward_propagation(X, parameters):
    """
    Implements the forward propagation for the model: LINEAR -> RELU -> LINEAR -> RELU -> LINEAR -> SOFTMAX
    
    Arguments:
    X -- input dataset placeholder, of shape (input size, number of examples)
    parameters -- python dictionary containing your parameters "W1", "b1", "W2", "b2", "W3", "b3"
                  the shapes are given in initialize_parameters

    Returns:
    Z3 -- the output of the last LINEAR unit
    """
    
    # Retrieve the parameters from the dictionary "parameters" 
    W1 = parameters['W1']
    b1 = parameters['b1']
    W2 = parameters['W2']
    b2 = parameters['b2']
    W3 = parameters['W3']
    b3 = parameters['b3']
    
                                                                                    # Numpy Equivalents:
    Z1 = tf.add(tf.matmul(W1, X), b1)                                              # Z1 = np.dot(W1, X) + b1
    A1 = tf.nn.relu(Z1)                                                             # A1 = relu(Z1)
    Z2 = tf.add(tf.matmul(W2, A1), b2)                                              # Z2 = np.dot(W2, a1) + b2
    A2 = tf.nn.relu(Z2)                                                             # A2 = relu(Z2)
    Z3 = tf.add(tf.matmul(W3, A2), b3)                                              # Z3 = np.dot(W3,Z2) + b3
    
    
    return Z3

In [190]:
def compute_cost(Z3, Y):
    """
    Computes the cost
    
    Arguments:
    Z3 -- output of forward propagation (output of the last LINEAR unit), of shape (6, number of examples)
    Y -- "true" labels vector placeholder, same shape as Z3
    
    Returns:
    cost - Tensor of the cost function
    """
    
    # to fit the tensorflow requirement for tf.nn.softmax_cross_entropy_with_logits(...,...)
    logits = tf.transpose(Z3)
    labels = tf.transpose(Y)
    
    
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=labels))
    
    
    return cost

In [191]:
def random_mini_batches(X, Y, minibatch_size, seed=0):
    """
    Creates random mini_batches for gradient descent
    
    Arguments:
    X --- training examples
    Y -- training labels
    """
    m = X.shape[1]
    np.random.seed(seed)
    permutations = list(np.random.permutation(m))
    
    ##shuffle the training set
    X_shuffled = X[:, permutations]
    Y_shuffled = Y[:, permutations].reshape(Y.shape[0], m)
    
    mini_batches = []
    
    num_mini_batches = int(math.floor(m/minibatch_size))
    
    for k in range(num_mini_batches):
        
        minibatch_X = X_shuffled[:, k * minibatch_size: k * minibatch_size + minibatch_size ]
        minibatch_Y = Y_shuffled[:, k * minibatch_size: k * minibatch_size + minibatch_size ]
        
        minibatch = (minibatch_X, minibatch_Y)
        mini_batches.append(minibatch)
        
        
    if m % minibatch_size != 0:
        minibatch_X = X_shuffled[:, num_mini_batches * minibatch_size: m]
        minibatch_Y = Y_shuffled[:, num_mini_batches * minibatch_size: m]
        minibatch = (minibatch_X, minibatch_Y)
        mini_batches.append(minibatch)
    
    return mini_batches

In [192]:
def model(X_train, Y_train, X_test, Y_test, learning_rate = 0.0001,
          num_epochs = 1500, minibatch_size = 32, print_cost = True):
    """
    Implements a three-layer tensorflow neural network: LINEAR->RELU->LINEAR->RELU->LINEAR->SOFTMAX.
    
    Arguments:
    X_train -- training set, of shape (input size = 12288, number of training examples = 1080)
    Y_train -- test set, of shape (output size = 6, number of training examples = 1080)
    X_test -- training set, of shape (input size = 12288, number of training examples = 120)
    Y_test -- test set, of shape (output size = 6, number of test examples = 120)
    learning_rate -- learning rate of the optimization
    num_epochs -- number of epochs of the optimization loop
    minibatch_size -- size of a minibatch
    print_cost -- True to print the cost every 100 epochs
    
    Returns:
    parameters -- parameters learnt by the model. They can then be used to predict.
    """
    
    ops.reset_default_graph()                         # to be able to rerun the model without overwriting tf variables
    tf.set_random_seed(1)                             # to keep consistent results
    seed = 3                                          # to keep consistent results
    (n_x, m) = X_train.shape                          # (n_x: input size, m : number of examples in the train set)
    n_y = Y_train.shape[0]                            # n_y : output size
    costs = []                                        # To keep track of the cost
    
    # Create Placeholders of shape (n_x, n_y)
    ### START CODE HERE ### (1 line)
    X, Y = create_placeholders(n_x, n_y)
    ### END CODE HERE ###

    # Initialize parameters
    ### START CODE HERE ### (1 line)
    parameters = initialize_parameters()
    ### END CODE HERE ###
    
    # Forward propagation: Build the forward propagation in the tensorflow graph
    ### START CODE HERE ### (1 line)
    Z3 = forward_propagation(X, parameters)
    ### END CODE HERE ###
    
    # Cost function: Add cost function to tensorflow graph
    ### START CODE HERE ### (1 line)
    cost = compute_cost(Z3, Y)
    ### END CODE HERE ###
    
    # Backpropagation: Define the tensorflow optimizer. Use an AdamOptimizer.
    ### START CODE HERE ### (1 line)
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
    ### END CODE HERE ###
    
    # Initialize all the variables
    init = tf.global_variables_initializer()

    # Start the session to compute the tensorflow graph
    with tf.Session() as sess:
        
        # Run the initialization
        sess.run(init)
        
        # Do the training loop
        for epoch in range(num_epochs):

            epoch_cost = 0.                       # Defines a cost related to an epoch
            num_minibatches = int(m / minibatch_size) # number of minibatches of size minibatch_size in the train set
            seed = seed + 1
            minibatches = random_mini_batches(X_train, Y_train, minibatch_size, seed)

            for minibatch in minibatches:

                # Select a minibatch
                (minibatch_X, minibatch_Y) = minibatch
                
                # IMPORTANT: The line that runs the graph on a minibatch.
                # Run the session to execute the "optimizer" and the "cost", the feedict should contain a minibatch for (X,Y).
                ### START CODE HERE ### (1 line)
                _ , minibatch_cost = sess.run([optimizer, cost], feed_dict={X: minibatch_X, Y: minibatch_Y})
                ### END CODE HERE ###
                
                epoch_cost += minibatch_cost / num_minibatches

            # Print the cost every epoch
            if print_cost == True and epoch % 100 == 0:
                print ("Cost after epoch %i: %f" % (epoch, epoch_cost))
            if print_cost == True and epoch % 5 == 0:
                costs.append(epoch_cost)
                
        # plot the cost
        plt.plot(np.squeeze(costs))
        plt.ylabel('cost')
        plt.xlabel('iterations (per tens)')
        plt.title("Learning rate =" + str(learning_rate))
        plt.show()

        # lets save the parameters in a variable
        parameters = sess.run(parameters)
        print ("Parameters have been trained!")

        # Calculate the correct predictions
        correct_prediction = tf.equal(tf.argmax(Z3), tf.argmax(Y))

        # Calculate accuracy on the test set
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

        print ("Train Accuracy:", accuracy.eval({X: X_train, Y: Y_train}))
        print ("Test Accuracy:", accuracy.eval({X: X_test, Y: Y_test}))
        
        return parameters

In [194]:
X_train, Y_train, X_test, Y_test = X_train[:, :8177], hotted[:, :8177], X_train[:, 8177:], hotted[:,8177:]

In [ ]:
model(X_train, Y_train, X_test, Y_test, learning_rate=0.001, num_epochs=1500, minibatch_size=2000, print_cost=True)

In [23]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

In [147]:
def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

In [148]:
def conv2d(x,W):
    return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding='SAME')

In [149]:
def max_pool_2X2(x):
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

In [196]:
W_conv1 = weight_variable([5,5,1,32])

In [197]:
b_conv1 = bias_variable([32])

In [270]:
x = tf.placeholder(tf.float32, shape=[784, None])
x_image = tf.reshape(x, [-1, 28, 28, 1])

In [271]:
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)

In [272]:
h_pool1 = max_pool_2X2(h_conv1)

In [273]:
W_conv2 = weight_variable([5,5,32,64])

In [274]:
b_conv2 = bias_variable([64])

In [275]:
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)

In [276]:
h_pool2 = max_pool_2X2(h_conv2)

In [277]:
W_fc1 = weight_variable([7 * 7 * 64, 1024])

In [278]:
b_fc1 = bias_variable([1024])

In [279]:
h_pool2_flat = tf.reshape(h_pool2, [-1,7 * 7 * 64])

In [280]:
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

In [281]:
keep_prob = tf.placeholder(tf.float32)

In [282]:
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

In [283]:
y_ = tf.placeholder(tf.float32, shape=[120, None])

In [284]:
#readout variable

W_fc2 = weight_variable([1024, 120])
b_fc2 = bias_variable([120])

In [285]:
y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

In [286]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))

In [287]:
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

In [288]:
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))

In [289]:
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [290]:
def cnn_model(X_train, Y_train, X_test, Y_test, num_epochs=20000, minibatch_size = 50):
    
    m = X_train.shape[1]
    
    seed = 3
    
    with tf.Session() as sess:
        
        sess.run(tf.global_variables_initializer())
        
        for epoch in range(num_epochs):
            
            seed = seed + 1
            
            minibatches = random_mini_batches(X_train, Y_train, minibatch_size= minibatch_size, seed=seed)
            
            
            for minibatch in minibatches:
                
                (minibatch_x, minibatch_y) = minibatch
                
                assert minibatch_x.shape[1] == minibatch_size
                
                if epoch % 100 == 0:
                    train_accuracy = accuracy.eval(feed_dict={x: minibatch_x, y_: minibatch_y, keep_prob: 1.0})
                    print "step {}, training accuracy {}".format(epoch, train_accuracy)
                train_step.run(feed_dict={x: minibatch_x, y: minibatch_y, keep_prob: 0.5})
                
        print "test accuracy is {}".format(accuracy.eval(feed_dict={x: X_test, y: Y_test, keep_prob: 1.0}))
                